## 1. Interfaces do Domínio

### CacheStore Protocol

Interface que define o contrato para operações de cache:

In [ ]:
// src/domain/protocols/cache-store.ts
export interface CacheStore {
  save: (key: string, value: any) => Promise<void>;
  delete: (key: string) => Promise<void>;
  fetch: (key: string) => Promise<any | null>;
}

### Purchase Model

Define a estrutura de dados para compras:

In [ ]:
// src/domain/models/purchase.ts
export interface PurchaseModel {
  id: string;
  title?: string;
  price: number;
  quantity: number;
  date: Date;
}

export type PurchasesModel = PurchaseModel[];

### SavePurchases Use Case Interface

Define o contrato do use case de salvar compras:

In [ ]:
// src/domain/usecases/save-purchases.ts
import type { PurchasesModel } from '@/domain/models/purchase.js';

export interface SavePurchases {
  save: (purchases: PurchasesModel) => Promise<void>;
}

## 2. Padrão de Testes - Test Double (Spy)

### O que é um Test Double?

Um **Test Double** é um substituto de um objeto real usado em testes. Os tipos principais são:

- **Stub**: retorna valores pré-configurados
- **Mock**: verifica se foi chamado com parâmetros corretos
- **Spy**: registra chamadas mas executa comportamento real
- **Fake**: tem lógica funcional simplificada
- **Dummy**: apenas preenchimento

### CacheStoreSpy

Implementação de um Spy para rastrear chamadas de `CacheStore`:

In [ ]:
// tests/data/usecases/local-save-purchases.spec.ts
import type { CacheStore } from '@/domain/protocols/cache-store.js';

enum CacheStoreCalls {
  delete,
  save,
}

class CacheStoreSpy implements CacheStore {
  messages: Array<CacheStoreCalls> = [];

  async save(): Promise<void> {
    this.messages.push(CacheStoreCalls.save);
  }

  async delete(): Promise<void> {
    this.messages.push(CacheStoreCalls.delete);
  }

  async fetch(): Promise<any> {
    return null;
  }
}

## 3. Implementação do SUT (System Under Test)

**SUT** = System Under Test (o objeto que você está testando)

### LocalSavePurchases

Implementação concreta que salva compras em cache:

In [ ]:
// src/data/usecases/local-save-purchases.ts
import type { CacheStore } from '@/domain/protocols/cache-store.js';
import type { SavePurchases } from '@/domain/usecases/save-purchases.js';

export class LocalSavePurchases implements SavePurchases {
  constructor(private readonly cacheStore: CacheStore) {}

  async save(): Promise<void> {
    await this.cacheStore.delete();
    await this.cacheStore.save();
  }
}

## 4. Testes Unitários

### Padrão: SUT Factory

Factory pattern para criar instâncias de teste:

In [ ]:
function SutFactory() {
  const cacheStore = new CacheStoreSpy();
  const sut = new LocalSavePurchases(cacheStore);
  return { cacheStore, sut };
}

### Teste 1: Não chama métodos na inicialização

Valida que o SUT não executa ações automaticamente:

In [ ]:
describe('LocalSavePurchases', () => {
  test('Should not call any method when initialized sut', () => {
    const { cacheStore } = SutFactory();
    expect(cacheStore.messages).toEqual([]);
  });
});

### Teste 2: Delete antes de Save

Valida que a ordem correta de operações é mantida:

In [ ]:
test('Should delete before save new cache', async () => {
  const { cacheStore, sut } = SutFactory();
  await sut.save('purchases');
  expect(cacheStore.messages).toEqual([
    CacheStoreCalls.delete,
    CacheStoreCalls.save,
  ]);
});

## 5. Conceitos Chave

### AAA Pattern (Arrange-Act-Assert)

```typescript
// Arrange: Prepara os dados e objetos
const { cacheStore, sut } = SutFactory();

// Act: Executa a ação
await sut.save('purchases');

// Assert: Verifica o resultado
expect(cacheStore.messages).toEqual([...]);
```

### Responsabilidades de Teste

- **Testes de Save**: Verificam que dados são persistidos corretamente
- **Testes de Load**: Verificam que dados são recuperados corretamente
- **Testes de Delete**: Verificam que cache é limpo
- **Testes de integração**: Verificam fluxo end-to-end (save → load)

### O que NÃO fazer

❌ **Mockar o SUT**: Você testa o comportamento real, não um substituto  
❌ **Mockar múltiplas dependências**: Teste uma dependência por vez  
❌ **Misturar responsabilidades**: Save testa save, Load testa load  

### O que fazer

✅ **Mockar apenas as dependências externas**: CacheStore, Database, APIs  
✅ **Usar Spies para rastrear chamadas**: Verificar ordem, parâmetros  
✅ **Uma responsabilidade por teste**: Um comportamento = um teste

## 6. Próximos Passos

1. **Implementar LoadPurchases**: Use case para carregar compras do cache
2. **Testes de integração**: Combinar save + load com InMemoryCacheStore
3. **Tratamento de erros**: Testes para casos de falha (cache vazio, dados inválidos)
4. **Serialização**: Validar conversão Date ↔ string ao persistir

## 7. Referências

- **Clean Architecture**: Separação de camadas (domain, data, presentation)
- **TDD**: Red-Green-Refactor cycle
- **Test Doubles**: Padrões para isolamento em testes
- **Vitest**: Framework de testes usado no projeto